### Compute variant associated IR and FR (normalization: v4 with lags) 
### Final step
### Update: from 12/15 to 1/16/22

### Import IR, PP and lagged_FR - assemble them into a single file each.

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from datetime import datetime as dt
from datetime import timedelta
from scipy.signal import lfilter
import pickle as pickle
import sys
import gc

In [2]:
# load required files - lagged FR

# CHANGE FILENAMES - LOAD ALL OUTPUTS YOU GENERATED

with open(f"../Input_files/lagged_FR1_011622.pkl",'rb') as file:
    lagged_FR1 = pickle.load(file)
    
with open(f"../Input_files/lagged_FR2_011622.pkl",'rb') as file:
    lagged_FR2 = pickle.load(file)
    

In [3]:
list_dates=list(lagged_FR1.keys())

lagged_FR=lagged_FR1

for date in tqdm(list_dates[700:717]):
    
    for var in lagged_FR[date].keys():
        lagged_FR[date][var]=lagged_FR2[date][var]  

# If you have more than 2 partial files, keep adding them to the aggregate list - for example, supposing you have two more partial files lagged_FR3 and lagged_FR4:

# for date in tqdm(list_dates[717:730]):   # CHANGE DATES
#    for var in lagged_FR[date].keys():\n",
#        lagged_FR[date][var]=lagged_FR3[date][var]
#
# for date in tqdm(list_dates[730:745]):  # CHANGE DATES
#     for var in lagged_FR[date].keys():\n",
#         lagged_FR[date][var]=lagged_FR4[date][var]

del lagged_FR1
del lagged_FR2 # delete all partial files
gc.collect()

100%|██████████| 17/17 [00:07<00:00,  2.33it/s]


0

In [4]:
# IR

# CHANGE FILENAMES - LOAD ALL OUTPUTS YOU GENERATED

with open(f"../Input_files/IR1_011622.pkl",'rb') as file:
    IR1 = pickle.load(file)
    
with open(f"../Input_files/IR2_011622.pkl",'rb') as file:
    IR2 = pickle.load(file)
    

In [5]:
IR=IR1

for date in tqdm(list_dates[700:717]):
    for var in IR[date].keys():
        IR[date][var]=IR2[date][var]
        
# SAME AS ABOVE - EXTEND TO ALL PARTIAL FILES AS NEEDED
        
del IR1
del IR2
gc.collect()

100%|██████████| 17/17 [00:08<00:00,  1.96it/s]


0

In [6]:
# PP

# CHANGE FILENAMES - LOAD ALL OUTPUTS YOU GENERATED

with open(f"../Input_files/PP1_011622.pkl",'rb') as file:
    PP1 = pickle.load(file)
    
with open(f"../Input_files/PP2_011622.pkl",'rb') as file:
    PP2 = pickle.load(file)
    
    
del PP1
del PP2

gc.collect()

In [7]:
PP=PP1

# SAME AS ABOVE - EXTEND TO ALL PARTIAL FILES AS NEEDED

for date in tqdm(list_dates[700:717]):
    for var in PP[date].keys():
        PP[date][var]=PP2[date][var]
        

100%|██████████| 17/17 [00:09<00:00,  1.87it/s]


In [9]:
# save 

# CHANGE DATES
with open(f"../Input_files/lagged_FR_12_15_21_01_16_22.pkl",'wb') as file:
    pickle.dump(lagged_FR,file,protocol=pickle.HIGHEST_PROTOCOL)
# CHANGE DATES
with open(f"../Input_files/IR_12_15_21_01_16_22.pkl",'wb') as file:
    pickle.dump(IR,file,protocol=pickle.HIGHEST_PROTOCOL)
# CHANGE DATES
with open(f"../Input_files/PP_12_15_21_01_16_22.pkl",'wb') as file:
    pickle.dump(PP,file,protocol=pickle.HIGHEST_PROTOCOL)


In [2]:
# If you just need to load the files before for the final table writing step, uncomment this cell and execute

#with open(f"../Input_files/lagged_FR_12_15_21_01_16_22.pkl",'rb') as file:
#    lagged_FR = pickle.load(file)
    
#with open(f"../Input_files/IR_12_15_21_01_16_22.pkl",'rb') as file:
#    IR = pickle.load(file)
    
#with open(f"../Input_files/PP_12_15_21_01_16_22.pkl",'rb') as file:
#    PP = pickle.load(file)
    
#list_dates=list(lagged_FR.keys())

In [3]:
with open(f"../Input_files/aggregated_mutations_window00_12_16_21_01_16_22.pkl",'rb') as file:
    aggregated_mutations_window = pickle.load(file)

In [4]:
# Finally, assemble and save the output tables for GPR.

# CHANGE INDEXES - these will be the same as in jh_cncb_update_processing_I.py, line 58 (AFTER YOU UPDATED THE INDEXES TO CURRENT UPDATE)
for date in tqdm(list_dates[684:717]):
    unique_muts = pd.DataFrame(list(aggregated_mutations_window[date].keys()),columns=['descriptor'])
    unique_muts = pd.DataFrame.join(unique_muts,pd.DataFrame(unique_muts['descriptor'].str.split(',').to_list())) #assign columns with parsed descriptor
    unique_muts.set_index('descriptor',inplace=True)
    unique_muts[0] = pd.to_numeric(unique_muts[0])
    unique_muts[1] = pd.to_numeric(unique_muts[1])
    unique_muts.sort_values([0,1],inplace=True)
    unique_muts.columns = ['Start','End','Ref','Alt','VEP','Variant Type','Ref_AA','Alt_AA','AA']

    counts = {}
    num_countries = {}
    infection_rate = {}
    fatality_rate = {}
    counted_countries = {}

    for desc in unique_muts.index:
        counts[desc] = len(aggregated_mutations_window[date][desc])
        num_countries[desc] = len(set(aggregated_mutations_window[date][desc]))
        ir_sum=sum(IR[date][desc])
        pp_sum=sum(PP[date][desc])
        if pp_sum==0:
            infection_rate[desc] = ir_sum
        else:
            infection_rate[desc] = ir_sum/pp_sum #/counts[desc]
        #infection_rate[desc] = np.mean(aggregated_mutations_window[date][desc][1]) #since accumulated over countries before denomenator of mean is wrong
        if counts[desc]==0:
            fatality_rate[desc] = sum(lagged_FR[date][desc])
        else:
            fatality_rate[desc] = sum(lagged_FR[date][desc])/counts[desc]#/counts[desc]
        #fatality_rate[desc] = np.mean(aggregated_mutations_window[date][desc][2]) #since accumulated over countries before denomenator of mean is wrong
        counted_countries[desc] = dict(Counter(aggregated_mutations_window[date][desc]))

    unique_muts['counts'] = unique_muts.index.to_series().map(counts)
    unique_muts['countries'] = unique_muts.index.to_series().map(num_countries)
    unique_muts['infection_rate'] = unique_muts.index.to_series().map(infection_rate)
    unique_muts['fatality_rate'] = unique_muts.index.to_series().map(fatality_rate)
    unique_muts['counted_countries'] = unique_muts.index.to_series().map(counted_countries)
    unique_muts.index = unique_muts.index.str.split(',').str[0:4].str.join('_')
    unique_muts.sort_values('counts',ascending=False)
    # CREATE A FOLDER FOR THE OUTPUT IN ../Output Files: cumulative_daily_AF_v4_lag_<your date>
    # CHANGE FOLDER NAME BELOW TO THE FOLDER YOU CREATED
    unique_muts.to_csv(f"../Output Files/cumulative_daily_AF_v4_lag_01_16_22/{date.strftime('%m_%d_%y')}.csv")

100%|██████████| 33/33 [45:42<00:00, 83.10s/it]
